In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.grid_search import GridSearchCV
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
#获取数据
Teams = pd.read_csv("../input/Teams.csv")  #球队id和名字
Seasons = pd.read_csv("../input/Seasons.csv")  #每个赛季wxyz 和 地域对应的关系
TourneySeeds = pd.read_csv("../input/TourneySeeds.csv")  #每个赛季季后赛的种子队
TourneyDetailedResults = pd.read_csv("../input/TourneyDetailedResults.csv")  #每个赛季季后赛每场比赛的具体结果
TourneySlots = pd.read_csv("../input/TourneySlots.csv")  #每个赛季季后赛对阵情况，考虑有playin比赛
RegularSeasonCompactResults = pd.read_csv("../input/RegularSeasonCompactResults.csv")  #每个赛季常规赛比赛结果
RegularSeasonDetailedResults = pd.read_csv("../input/RegularSeasonDetailedResults.csv")  #每个赛季常规赛比赛详细结果
TourneyCompactResults = pd.read_csv("../input/TourneyCompactResults.csv")  #每个赛季季后赛每场比赛的基本结果

TourneySlots.head()

In [ ]:
#利用整体的数据，比如球队每个赛季的胜率，球队平均每场得分，球队平均每场失分，投篮命中率，三分命中率，罚球命中率，进攻篮板，防守篮板，助攻，失误，抢断，盖帽，犯规
#这个主要是使用RegularSeasonDetailedResults和RegularSeasonCompactResults

#合并赢球和输球，获取每个球队球场基础数据
df1=RegularSeasonDetailedResults[['Season','Wteam','Wscore','Numot','Wfgm','Wfga','Wfgm3','Wfga3','Wftm','Wfta','Wor','Wdr','Wast','Wto','Wstl','Wblk','Wpf']]
df2=RegularSeasonDetailedResults[['Season','Lteam','Lscore','Numot','Lfgm','Lfga','Lfgm3','Lfga3','Lftm','Lfta','Lor','Ldr','Last','Lto','Lstl','Lblk','Lpf']]
df3 = df2
df3.columns = ['Season','Wteam','Wscore','Numot','Wfgm','Wfga','Wfgm3','Wfga3','Wftm','Wfta','Wor','Wdr','Wast','Wto','Wstl','Wblk','Wpf']
pds = df1.append(df3,ignore_index=True)
detailavg = pds.groupby(['Season', 'Wteam'])[['Wscore','Numot','Wfgm','Wfga','Wfgm3','Wfga3','Wftm','Wfta','Wor','Wdr','Wast','Wto','Wstl','Wblk','Wpf']].mean()
#detailavg = RegularSeasonDetailedResults.groupby(['Season', 'Wteam']).mean()
detailavg.tail()

#获取非详细的数据
df1=RegularSeasonCompactResults[['Season','Wteam','Wscore','Numot']]
df2=RegularSeasonCompactResults[['Season','Lteam','Lscore','Numot']]
df3 = df2
df3.columns = ['Season','Wteam','Wscore','Numot']
pds = df1.append(df3,ignore_index=True)
compactavg = pds.groupby(['Season', 'Wteam'])[['Wscore','Numot']].mean()
#detailavg = RegularSeasonDetailedResults.groupby(['Season', 'Wteam']).mean()
compactavg.tail()

In [ ]:
#算出每只球队主客场数据
##获取每个球队主场数据
df1=RegularSeasonDetailedResults[RegularSeasonDetailedResults.Wloc == 'H'][['Season','Wteam','Wscore','Numot','Wfgm','Wfga','Wfgm3','Wfga3','Wftm','Wfta','Wor','Wdr','Wast','Wto','Wstl','Wblk','Wpf']]
df2=RegularSeasonDetailedResults[['Season','Lteam','Lscore','Numot','Lfgm','Lfga','Lfgm3','Lfga3','Lftm','Lfta','Lor','Ldr','Last','Lto','Lstl','Lblk','Lpf']][RegularSeasonDetailedResults.Wloc == 'A']
df3 = df2
df3.columns = ['Season','Wteam','Wscore','Numot','Wfgm','Wfga','Wfgm3','Wfga3','Wftm','Wfta','Wor','Wdr','Wast','Wto','Wstl','Wblk','Wpf']
pds = df1.append(df3,ignore_index=True)
zhudetailavg = pds.groupby(['Season', 'Wteam'])[['Wscore','Numot','Wfgm','Wfga','Wfgm3','Wfga3','Wftm','Wfta','Wor','Wdr','Wast','Wto','Wstl','Wblk','Wpf']].mean()
#detailavg = RegularSeasonDetailedResults.groupby(['Season', 'Wteam']).mean()
zhudetailavg.tail()

df1=RegularSeasonCompactResults[RegularSeasonCompactResults.Wloc == 'H'][['Season','Wteam','Wscore','Numot']]
df2=RegularSeasonCompactResults[RegularSeasonCompactResults.Wloc == 'A'][['Season','Lteam','Lscore','Numot']]
df3 = df2
df3.columns = ['Season','Wteam','Wscore','Numot']
pds = df1.append(df3,ignore_index=True)
zhucompactavg = pds.groupby(['Season', 'Wteam'])[['Wscore','Numot']].mean()
#detailavg = RegularSeasonDetailedResults.groupby(['Season', 'Wteam']).mean()
#获取每个球队客场数据
df1=RegularSeasonDetailedResults[['Season','Wteam','Wscore','Numot','Wfgm','Wfga','Wfgm3','Wfga3','Wftm','Wfta','Wor','Wdr','Wast','Wto','Wstl','Wblk','Wpf']][RegularSeasonDetailedResults.Wloc == 'A']
df2=RegularSeasonDetailedResults[['Season','Lteam','Lscore','Numot','Lfgm','Lfga','Lfgm3','Lfga3','Lftm','Lfta','Lor','Ldr','Last','Lto','Lstl','Lblk','Lpf']][RegularSeasonDetailedResults.Wloc == 'H']
df3 = df2
df3.columns = ['Season','Wteam','Wscore','Numot','Wfgm','Wfga','Wfgm3','Wfga3','Wftm','Wfta','Wor','Wdr','Wast','Wto','Wstl','Wblk','Wpf']
pds = df1.append(df3,ignore_index=True)
kedetailavg = pds.groupby(['Season', 'Wteam'])[['Wscore','Numot','Wfgm','Wfga','Wfgm3','Wfga3','Wftm','Wfta','Wor','Wdr','Wast','Wto','Wstl','Wblk','Wpf']].mean()
#detailavg = RegularSeasonDetailedResults.groupby(['Season', 'Wteam']).mean()
kedetailavg.tail()

df1=RegularSeasonCompactResults[['Season','Wteam','Wscore','Numot']][RegularSeasonCompactResults.Wloc == 'A']
df2=RegularSeasonCompactResults[['Season','Lteam','Lscore','Numot']][RegularSeasonCompactResults.Wloc == 'H']
df3 = df2
df3.columns = ['Season','Wteam','Wscore','Numot']
pds = df1.append(df3,ignore_index=True)
kecompactavg = pds.groupby(['Season', 'Wteam'])[['Wscore','Numot']].mean()

#获取每个球队中立场地数据


In [ ]:
#利用相互之间的对战数据
df1=RegularSeasonDetailedResults[['Season','Wteam','Wscore','Numot','Wfgm','Wfga','Wfgm3','Wfga3','Wftm','Wfta','Wor','Wdr','Wast','Wto','Wstl','Wblk','Wpf','Lteam','Lscore','Numot','Lfgm','Lfga','Lfgm3','Lfga3','Lftm','Lfta','Lor','Ldr','Last','Lto','Lstl','Lblk','Lpf']]
df2=RegularSeasonDetailedResults[['Season','Lteam','Lscore','Numot','Lfgm','Lfga','Lfgm3','Lfga3','Lftm','Lfta','Lor','Ldr','Last','Lto','Lstl','Lblk','Lpf','Wteam','Wscore','Numot','Wfgm','Wfga','Wfgm3','Wfga3','Wftm','Wfta','Wor','Wdr','Wast','Wto','Wstl','Wblk','Wpf']]
df3 = df2
df3.columns = ['Season','Wteam','Wscore','Numot','Wfgm','Wfga','Wfgm3','Wfga3','Wftm','Wfta','Wor','Wdr','Wast','Wto','Wstl','Wblk','Wpf','Lteam','Lscore','Numot','Lfgm','Lfga','Lfgm3','Lfga3','Lftm','Lfta','Lor','Ldr','Last','Lto','Lstl','Lblk','Lpf']
pds = df1.append(df3,ignore_index=True)
xianghudetailavg = pds.groupby(['Season','Wteam','Lteam']).mean()[['Wscore','Numot','Wfgm','Wfga','Wfgm3','Wfga3','Wftm','Wfta','Wor','Wdr','Wast','Wto','Wstl','Wblk','Wpf']]
xianghudetailavg.tail()

df1=RegularSeasonCompactResults[['Season','Wteam','Wscore','Numot','Lteam','Lscore']]
df2=RegularSeasonCompactResults[['Season','Lteam','Lscore','Numot','Wteam','Wscore']]
df3 = df2
df3.columns = ['Season','Wteam','Wscore','Numot','Lteam','Lscore']
pds = df1.append(df3,ignore_index=True)
xianghucompactavg = pds.groupby(['Season', 'Wteam','Lteam']).mean()[['Wscore','Numot']]
xianghucompactavg.tail()

In [ ]:
#利用相互之间的主客场对战数
